# Danish demand and supply 1960-2022

**Imports**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

# for import of DST
import requests
from IPython.display import display
from io import StringIO
# for fetching data
import pandas_datareader
from dstapi import DstApi

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# user written modules
import Final_dataproject as model
data = model

**Fetching data**

We first need to import our data for use in the project. First we look at the variables in our dataset "NAN1"

In [2]:
nan1 = DstApi('NAN1')
nan1.tablesummary(language='en')

Table NAN1: Demand and supply by transaction, price unit and time
Last update: 2023-03-31T08:00:00


,variable name,# values,First value,First value label,Last value,Last value label,Time variable
0,TRANSAKT,31,B1GQK,B.1*g Gross domestic product,EMPM_DC,"Total employment (1,000 persons)",False
1,PRISENHED,6,V_M,"Current prices, (bill. DKK.)",LAN_C,"Pr. capita, 2010-prices, chained values, (1000...",False
2,Tid,57,1966,1966,2022,2022,True


In [3]:
#Look at each potential variabel value for each variabel
display(nan1.variable_levels('TRANSAKT',language='en'))
display(nan1.variable_levels('PRISENHED',language='en'))
#display(nan1.variable_levels('Tid',language='en'))

,id,text
0,B1GQK,B.1*g Gross domestic product
1,P7K,P.7 Imports of goods and services
2,P71K,P.71 Import of goods
3,P72K,P.72 Import of services
4,TFSPR,Supply
5,P6D,P.6 Exports of goods and services
6,P61D,P.61 Export of goods
7,P62D,P.62 Export of services
8,P31S1MD,P.31 Private consumption
9,P31S14D,P.31 Household consumption expenditure


,id,text
0,V_M,"Current prices, (bill. DKK.)"
1,LAN_M,"2010-prices, chained values, (bill. DKK.)"
2,L_V,Period-to-period real growth (per cent)
3,V_C,"Pr. capita. Current prices, (1000 DKK.)"
4,L_VB,"Contribution to GDP growth, (percentage point)"
5,LAN_C,"Pr. capita, 2010-prices, chained values, (1000..."


I now select the variable values I want to use for the analysis

In [4]:
params = nan1._define_base_params(language='en')
params

{'table': 'nan1',
 'format': 'BULK',
 'lang': 'en',
 'variables': [{'code': 'TRANSAKT', 'values': ['*']},
  {'code': 'PRISENHED', 'values': ['*']},
  {'code': 'Tid', 'values': ['*']}]}

In [5]:
# choose the data to keep
params = {
    'table': 'nan1',
 'format': 'BULK',
 'lang': 'en',
 'variables': [{'code': 'TRANSAKT', 'values': ['B1GQK','P7K','TFSPR','P6D','P31S1MD','P3S13D','P5GD','TFUPR']},
  {'code': 'PRISENHED', 'values': ['LAN_M']},
  {'code': 'Tid', 'values': ['*']}]}



In [6]:
#Rename variable values
nan1_api = nan1.get_data(params=params)
rename = {'B.1*g Gross domestic product':'Y',
          'P.3 Government consumption expenditure':'G',
          'P.31 Private consumption':'C',
          'P.5g Gross capital formation':'I',
          'P.6 Exports of goods and services':'X',
          'P.7 Imports of goods and services':'M',
          'Final demand':'Demand'
}
nan1_api['TRANSAKT']=nan1_api['TRANSAKT'].replace(rename)
nan1_api.sort_values(by=['TID','TRANSAKT'], inplace=True)
nan1_api.head(8)

,TRANSAKT,PRISENHED,TID,INDHOLD
74,C,"2010-prices, chained values, (bill. DKK.)",1966,372.8
79,Demand,"2010-prices, chained values, (bill. DKK.)",1966,780.4
73,G,"2010-prices, chained values, (bill. DKK.)",1966,171.3
75,I,"2010-prices, chained values, (bill. DKK.)",1966,127.6
77,M,"2010-prices, chained values, (bill. DKK.)",1966,112.2
78,Supply,"2010-prices, chained values, (bill. DKK.)",1966,780.4
76,X,"2010-prices, chained values, (bill. DKK.)",1966,123.6
72,Y,"2010-prices, chained values, (bill. DKK.)",1966,702.4


In [7]:
#Save the data in a csv file
nan1_api.to_csv('nan1_api.csv', index=False)

**Load, clean, save data**

We now that we have the data we need for analysis, we now clean it and make it ready

In [8]:
dkbal=pd.read_csv('nan1_api.csv')
#Remove 'PRISENHED' collumn
del dkbal['PRISENHED']

dkbal.head(9)

,TRANSAKT,TID,INDHOLD
0,C,1966,372.8
1,Demand,1966,780.4
2,G,1966,171.3
3,I,1966,127.6
4,M,1966,112.2
5,Supply,1966,780.4
6,X,1966,123.6
7,Y,1966,702.4
8,C,1967,397.9


In [9]:
dkbal.loc[dkbal['TRANSAKT']=='C']

,TRANSAKT,TID,INDHOLD
0,C,1966,372.8
8,C,1967,397.9
16,C,1968,412.2
24,C,1969,435.8
32,C,1970,446.6
40,C,1971,450.3
48,C,1972,450.6
56,C,1973,484.7
64,C,1974,473.7
72,C,1975,488.5


In [14]:
for variable in dkbal['TID']:
    dkbal[variable]=dkbal['INDHOLD']
dkbal

,TRANSAKT,TID,INDHOLD,1966,1967,1968,1969,1970,1971,1972,...,2021,2022,C,Demand,G,I,M,Supply,X,Y
0,C,1966,372.8,372.8,372.8,372.8,372.8,372.8,372.8,372.8,...,372.8,372.8,372.8,372.8,372.8,372.8,372.8,372.8,372.8,372.8
1,Demand,1966,780.4,780.4,780.4,780.4,780.4,780.4,780.4,780.4,...,780.4,780.4,780.4,780.4,780.4,780.4,780.4,780.4,780.4,780.4
2,G,1966,171.3,171.3,171.3,171.3,171.3,171.3,171.3,171.3,...,171.3,171.3,171.3,171.3,171.3,171.3,171.3,171.3,171.3,171.3
3,I,1966,127.6,127.6,127.6,127.6,127.6,127.6,127.6,127.6,...,127.6,127.6,127.6,127.6,127.6,127.6,127.6,127.6,127.6,127.6
4,M,1966,112.2,112.2,112.2,112.2,112.2,112.2,112.2,112.2,...,112.2,112.2,112.2,112.2,112.2,112.2,112.2,112.2,112.2,112.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451,I,2022,566.9,566.9,566.9,566.9,566.9,566.9,566.9,566.9,...,566.9,566.9,566.9,566.9,566.9,566.9,566.9,566.9,566.9,566.9
452,M,2022,1218.2,1218.2,1218.2,1218.2,1218.2,1218.2,1218.2,1218.2,...,1218.2,1218.2,1218.2,1218.2,1218.2,1218.2,1218.2,1218.2,1218.2,1218.2
453,Supply,2022,3478.5,3478.5,3478.5,3478.5,3478.5,3478.5,3478.5,3478.5,...,3478.5,3478.5,3478.5,3478.5,3478.5,3478.5,3478.5,3478.5,3478.5,3478.5
454,X,2022,1394.9,1394.9,1394.9,1394.9,1394.9,1394.9,1394.9,1394.9,...,1394.9,1394.9,1394.9,1394.9,1394.9,1394.9,1394.9,1394.9,1394.9,1394.9
